## Implement Quasiparticle Hamiltonian on a IonQ simulator

In [1]:
import qiskit
import numpy as np
from qiskit.quantum_info import SparsePauliOp

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from typing import Dict


Load matrix elements for $^{18}O$

In [3]:
data_onebody=np.load('data/matrix_elements_h_eff_2body/one_body_nn_sd.npz')
keys=data_onebody['keys']
values=data_onebody['values']

t_onebody={}

for a,key in enumerate(keys):
    t_onebody[tuple(key)]=values[a]

print(t_onebody)


{(0, 0): -9.3151, (0, 1): 0.6897999999999996, (0, 2): -0.4062999999999996, (0, 3): -0.900781889989654, (0, 4): -1.9314456782406446, (0, 5): 0.6017349640876247, (1, 0): 0.6897999999999996, (1, 1): -8.8615, (1, 2): 0.8598999999999998, (1, 3): 0.900781889989654, (1, 4): 1.1336192497488327, (1, 5): -1.3995613925794368, (2, 0): -0.4062999999999996, (2, 1): 0.8598999999999998, (2, 2): -9.145, (2, 3): -0.900781889989654, (2, 4): -0.7347060355029268, (2, 5): 1.7984746068253425, (3, 0): -0.900781889989654, (3, 1): 0.900781889989654, (3, 2): -0.900781889989654, (3, 3): -8.107099999999999, (3, 4): -0.7177133829043456, (3, 5): 0.7177133829043456, (4, 0): -1.9314456782406446, (4, 1): 1.1336192497488327, (4, 2): -0.7347060355029268, (4, 3): -0.7177133829043456, (4, 4): 3.2251000000000003, (4, 5): 0.9008999999999998, (5, 0): 0.6017349640876247, (5, 1): -1.3995613925794368, (5, 2): 1.7984746068253425, (5, 3): 0.7177133829043456, (5, 4): 0.9008999999999998, (5, 5): 3.2251000000000003}


In [ ]:
def get_hamiltonian(t_onebody:Dict, n_sites:int):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    XX_tuples=[]
    YY_tuples=[]
    Z_tuples=[]
    I_tuples=[]
    for (i,j),t_val in t_onebody.items():
    
        if i==j:
            Z_tuples.append(("Z", [i], 0.5*t_val))
            I_tuples.append(('I',[i],0.5*t_val)) 
        else:
            XX_tuples.append( ("XX", [i, j], 0.25*t_val) )
            YY_tuples.append(("YY", [i, j], 0.25*t_val) )
            
    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*XX_tuples, *YY_tuples,*Z_tuples,*I_tuples], num_qubits=n_sites)
    return hamiltonian.simplify()

Get the manybody basis

In [28]:
import numpy as np
from qiskit.quantum_info import Statevector

n_sites=6
basis_states = [Statevector.from_label(np.binary_repr(i, width=n_sites)) for i in range(2**n_sites)]

bitstring_to_index = {np.binary_repr(i, width=n_sites): i for i in range(2**n_sites)}
index_to_bitstring = {i: np.binary_repr(i, width=n_sites) for i in range(2**n_sites)}

In [29]:
print(index_to_bitstring)

{0: '000000', 1: '000001', 2: '000010', 3: '000011', 4: '000100', 5: '000101', 6: '000110', 7: '000111', 8: '001000', 9: '001001', 10: '001010', 11: '001011', 12: '001100', 13: '001101', 14: '001110', 15: '001111', 16: '010000', 17: '010001', 18: '010010', 19: '010011', 20: '010100', 21: '010101', 22: '010110', 23: '010111', 24: '011000', 25: '011001', 26: '011010', 27: '011011', 28: '011100', 29: '011101', 30: '011110', 31: '011111', 32: '100000', 33: '100001', 34: '100010', 35: '100011', 36: '100100', 37: '100101', 38: '100110', 39: '100111', 40: '101000', 41: '101001', 42: '101010', 43: '101011', 44: '101100', 45: '101101', 46: '101110', 47: '101111', 48: '110000', 49: '110001', 50: '110010', 51: '110011', 52: '110100', 53: '110101', 54: '110110', 55: '110111', 56: '111000', 57: '111001', 58: '111010', 59: '111011', 60: '111100', 61: '111101', 62: '111110', 63: '111111'}


In [33]:
hamiltonian_q=get_hamiltonian(t_onebody=t_onebody,n_sites=6)

print(hamiltonian_q)

print(hamiltonian_q.to_matrix(sparse=True))

index=1
effective_index=63-2**index

print(hamiltonian_q.to_matrix(sparse=True)[effective_index,effective_index])


SparsePauliOp(['IIIIXX', 'IIIXIX', 'IIXIIX', 'IXIIIX', 'XIIIIX', 'IIIXXI', 'IIXIXI', 'IXIIXI', 'XIIIXI', 'IIXXII', 'IXIXII', 'XIIXII', 'IXXIII', 'XIXIII', 'XXIIII', 'IIIIYY', 'IIIYIY', 'IIYIIY', 'IYIIIY', 'YIIIIY', 'IIIYYI', 'IIYIYI', 'IYIIYI', 'YIIIYI', 'IIYYII', 'IYIYII', 'YIIYII', 'IYYIII', 'YIYIII', 'YYIIII', 'IIIIIZ', 'IIIIZI', 'IIIZII', 'IIZIII', 'IZIIII', 'ZIIIII', 'IIIIII'],
              coeffs=[  0.6898    +0.j,  -0.4063    +0.j,  -0.90078189+0.j,  -1.93144568+0.j,
   0.60173496+0.j,   0.8599    +0.j,   0.90078189+0.j,   1.13361925+0.j,
  -1.39956139+0.j,  -0.90078189+0.j,  -0.73470604+0.j,   1.79847461+0.j,
  -0.71771338+0.j,   0.71771338+0.j,   0.9009    +0.j,   0.6898    +0.j,
  -0.4063    +0.j,  -0.90078189+0.j,  -1.93144568+0.j,   0.60173496+0.j,
   0.8599    +0.j,   0.90078189+0.j,   1.13361925+0.j,  -1.39956139+0.j,
  -0.90078189+0.j,  -0.73470604+0.j,   1.79847461+0.j,  -0.71771338+0.j,
   0.71771338+0.j,   0.9009    +0.j,  -4.65755   +0.j,  -4.43075   +0.j,
  -4.5725

#### Qiskit has the sick problem that the occupation value 1 becomes 0 (they define $S_z$ in the opposite way). Therefore, to select the Hamiltonian fixed in the one quasiparticle many-body basis we need to consider the conversion to the full bitstrig basis 

bitstring_index= 63- $2^{sp \; index}$

In [34]:
hamiltonian_q_single_particle={}

for a in range(n_sites):
    for b in range(n_sites):
        a_mb=63-2**a
        b_mb=63-2**b
        hamiltonian_q_single_particle[(a,b)]=hamiltonian_q.to_matrix(sparse=True)[a_mb,b_mb]

In [39]:
print(hamiltonian_q_single_particle[0,1]/2)

(0.6897999999999996+0j)
